In [18]:
import os
import pandas as pd
import json

# Define the base directory where all site folders are stored
base_dir = r"C:\Users\Sandy\Documents\Aeronet\Data"

# Loop through each site folder (GSFC, HU_IRB, SERC, etc.)
for site in os.listdir(base_dir):
    site_path = os.path.join(base_dir, site)
    
    # Skip if it's not a directory
    if not os.path.isdir(site_path):
        continue

    print(f"\n🔹 Processing site: {site}")

    # Define paths for Raw, Processed, and Metadata folders
    raw_dir = os.path.join(site_path, "Raw")
    processed_dir = os.path.join(site_path, "Processed")
    metadata_dir = os.path.join(site_path, "Metadata")

    # Create necessary directories if they don't exist
    os.makedirs(processed_dir, exist_ok=True)
    os.makedirs(metadata_dir, exist_ok=True)

    # Define metadata file path (single file per site)
    metadata_file_path = os.path.join(metadata_dir, f"{site}_metadata.json")
    site_metadata = {}  # Dictionary to store all metadata for this site

    # Process each CSV file in the Raw folder
    for filename in os.listdir(raw_dir):
        if filename.endswith(".csv"):  # Only process CSV files
            raw_file_path = os.path.join(raw_dir, filename)
            processed_file_path = os.path.join(processed_dir, filename.replace("raw", "processed"))

            # Extract year from filename
            year = "".join(filter(str.isdigit, filename.split("_")[2]))  # Extracts 2020, 2021, etc.

            print(f"  📂 Processing file: {filename} (Year: {year})")

            # Read the raw file
            with open(raw_file_path, "r", encoding="utf-8") as file:
                lines = file.readlines()

            # Identify where actual data starts
            data_start_index = None
            for i, line in enumerate(lines):
                if line.startswith("AERONET_Site,Date(dd:mm:yyyy)") or "Date(dd:mm:yyyy)" in line:
                    data_start_index = i
                    break

            if data_start_index is None:
                print(f"  ❌ No valid data found in {filename}. Skipping...")
                continue

            # Separate metadata and actual data
            metadata = "".join(lines[:data_start_index])
            data = "".join(lines[data_start_index:])

            # Store metadata in the dictionary under the year key
            site_metadata[year] = metadata.split("\n")

            # Save actual data as a clean CSV
            with open(processed_file_path, "w", encoding="utf-8") as data_file:
                data_file.write(data)
            print(f"  ✅ Processed data saved: {processed_file_path}")

            # Load the processed data into a DataFrame
            df = pd.read_csv(processed_file_path)

            # Standardize column names
            df.columns = df.columns.str.strip()

            # Add DateTime column at index 1
            df.insert(1, 'DateTime', pd.to_datetime(df['Date(dd:mm:yyyy)'] + ' ' + df['Time(hh:mm:ss)'], format='%d:%m:%Y %H:%M:%S'))

            # Replace -999 values with NaN
            df.replace(-999, float('nan'), inplace=True)

            # Drop columns where all values are NaN
            df.dropna(axis=1, how="all", inplace=True)

            # Drop Date and Time columns
            columns_to_drop = ["Date(dd:mm:yyyy)", "Time(hh:mm:ss)"]
            df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

            # Save the cleaned data
            df.to_csv(processed_file_path, index=False)
            print(f"  ✅ Cleaned data saved: {processed_file_path}")

    # Save the combined metadata for the site
    with open(metadata_file_path, "w", encoding="utf-8") as meta_file:
        json.dump(site_metadata, meta_file, indent=4)
    print(f"  ✅ Combined metadata saved for {site}: {metadata_file_path}")

print("\n✅ All sites processed successfully!")



🔹 Processing site: GSFC
  📂 Processing file: GSFC_raw_2020_AOD15_directsun.csv (Year: 2020)
  ✅ Processed data saved: C:\Users\Sandy\Documents\Aeronet\Data\GSFC\Processed\GSFC_processed_2020_AOD15_directsun.csv
  ✅ Cleaned data saved: C:\Users\Sandy\Documents\Aeronet\Data\GSFC\Processed\GSFC_processed_2020_AOD15_directsun.csv
  📂 Processing file: GSFC_raw_2020_AOD20_directsun.csv (Year: 2020)
  ✅ Processed data saved: C:\Users\Sandy\Documents\Aeronet\Data\GSFC\Processed\GSFC_processed_2020_AOD20_directsun.csv
  ✅ Cleaned data saved: C:\Users\Sandy\Documents\Aeronet\Data\GSFC\Processed\GSFC_processed_2020_AOD20_directsun.csv
  📂 Processing file: GSFC_raw_2021_AOD15_directsun.csv (Year: 2021)
  ✅ Processed data saved: C:\Users\Sandy\Documents\Aeronet\Data\GSFC\Processed\GSFC_processed_2021_AOD15_directsun.csv
  ✅ Cleaned data saved: C:\Users\Sandy\Documents\Aeronet\Data\GSFC\Processed\GSFC_processed_2021_AOD15_directsun.csv
  📂 Processing file: GSFC_raw_2021_AOD20_directsun.csv (Year: 2